In [12]:
from numpy import savetxt
# Read results

with open("/Users/mac/Google Drive/CausalMARL/res_MARL/2_sd15_T/vary_T_sd_15_small.txt", "rb") as fp:
    Ts1 = pickle.load(fp) # len = 2sd_R * 1 * 3_target * (3, 6)
with open("/Users/mac/Google Drive/CausalMARL/res_MARL/2_sd15_T/vary_T_sd_15_large.txt", "rb") as fp:
    Ts2 = pickle.load(fp) # len = 2sd_R * 1 * 3_target * (3, 6)

In [13]:
from _utility import * 
from weight import * 
from simu_funs import *  
from main import *
a = now()

In [19]:
npseed(1)
u_O = rnorm(100, 20, 5**2)
def DG_once(seed = 1, l = 5, T = 14 * 48, t_func = None, u_O_u_D = 0, 
            w_A = 1, w_O = 1, sd_R  = 1, sd_D = 1, sd_O = 1, 
            u_O = u_O, 
           TARGET = False, target_policy = None, T_burn_in = 100):  
    """ prepare data (fixed)
    """
    M_in_R  = True
    u_O_u_D = u_O_u_D
    
    poisO = True
    T = T + T_burn_in
    npseed(seed)
    N = l ** 2
    adj_mat = getAdjGrid(l)
    p_behav = 0.5
    # random errors for D and R
#     e_D = (rpoisson(1, (N, T)) - 1) * sd_D

    e_R = randn(N, T) * sd_R
    # initialization
    M = [runi(0, 1, N)] 
    R = []
    
    # weight in the definition of mismatch

    w_M = 0.5    # 0.8

    # O: the pattern (spatial distribution) of orders
    """ TUNE
    """
    if poisO == True:
        O = rpoisson(u_O, (T, N)).T    
#     else:
#         O = np.repeat(u_O, T).reshape(N, T) +  (rpoisson(sd_O, (N, T)) - sd_O) #randn(N,T) #/ 10
#         O[O < 0] = 0

    # Actions
    if TARGET: # target. fixed. 
        A = arr([[target_policy[i](None, random_choose = True) for j in range(T)] for i in range(N)])
    else: # behaviour policy: random
        A = rbin(1, p_behav, (N, T))
    
    # D: initial is the same with driver. then attract by the A and O. burn-in.
    """ TUNE
    """
    u_D = np.mean(u_O) - u_O_u_D
    D = [arr([u_D for i in range(N)])]
    
    """ MAIN: state trasition and reward calculation [no action selection]
    """
    n_neigh = np.sum(adj_mat,1)
    for t in range(1, T): 
        """ Drivers
        """
        ## attractions
        """ TUNE
        """
#         Attr_OD = w_O * (squeeze(O[:, t - 1]) / (1 + squeeze(D[t - 1])))
        Attr_OD = w_O * (squeeze(O[:, t - 1]) / (squeeze(D[t - 1])))
        Attr = np.exp(w_A * A[:, t - 1]) + squeeze(Attr_OD) ## * at first
        
        
#         print(uv(np.exp(w_A * A[:, t - 1])), uv(squeeze(Attr_OD)), max(squeeze(Attr_OD)))
        
        Attr_mat = np.repeat(Attr, N).reshape(N, N)
        Attr_adj = np.multiply(Attr_mat, adj_mat)
        Attr_neigh = np.sum(Attr_adj, 0)

        D_t = squeeze(Attr_adj.dot((D[t - 1] / squeeze(Attr_neigh)).reshape(-1, 1)))

        D.append(D_t)
        O_t = O[:, t] 
        
        """ Tune: M and R 
        """
        M_t = w_M * (1 - abs(D_t - O_t) / abs(1 + D_t + O_t)) + (1 - w_M) * M[t - 1]
        # R_{t - 1}
        if M_in_R:
            R_t_1 = M_t * np.minimum(D_t, O_t) + e_R[:, t] 
        else:
            R_t_1 = np.minimum(D_t, O_t) + e_R[:, t]
        
        M.append(M_t)
        R.append(R_t_1)
    R.append(R_t_1) # add one more? # new reward?

    """ organization
    """
    ## organization and burn-in; N * T
    R = arr(R).T[:, T_burn_in:]; D = arr(D).T[:, T_burn_in:]; M = arr(M).T[:, T_burn_in:]
    O = O[:, T_burn_in:]; A = A[:, T_burn_in:]
    
    ## reorganization
    data = []
    for i in range(N):
        data_i = []
        for t in range(T - T_burn_in):
            data_i.append([arr([O[i, t], D[i, t], M[i, t]]), A[i, t], R[i, t]])
            print(data_i[0], data_i[0][0].shape, "\n")
        data.append(data_i)
    
    return data, adj_mat, [[O, D, M], A, R]

In [20]:
DG_once()

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342] (3,) 

[array([134.    ,  83.1447,   0.69

([[[array([134.    ,  83.1447,   0.6994]), 0, 45.47588405481342],
   [array([143.    ,  66.3088,   0.6674]), 0, 27.22335978393115],
   [array([147.    ,  46.5891,   0.5757]), 0, 55.6791679858449],
   [array([137.    ,  82.8171,   0.6651]), 0, 30.823999850612246],
   [array([134.    ,  52.0322,   0.6134]), 0, 40.03612067639493],
   [array([136.    ,  63.6431,   0.6264]), 0, 40.36084932676547],
   [array([136.    ,  62.865 ,   0.6302]), 0, 49.61777913385403],
   [array([121.    ,  73.0564,   0.6922]), 1, 63.52730297914462],
   [array([143.    ,  87.7574,   0.7269]), 1, 53.55346828262714],
   [array([140.    ,  75.0486,   0.7131]), 0, 39.82706536848538],
   [array([132.    ,  61.1255,   0.674 ]), 1, 73.4923029845662],
   [array([142.    ,  98.0972,   0.746 ]), 0, 44.06493748278015],
   [array([119.    ,  60.7221,   0.7117]), 1, 56.76465864468883],
   [array([143.    ,  80.4279,   0.7165]), 0, 34.790529586511845],
   [array([136.    ,  53.6293,   0.6422]), 1, 68.3091061699221],
   [array([

In [14]:
arr([3,3,3]).shape

(3,)

In [9]:
with open("/Users/mac/Google Drive/CausalMARL/res_MARL/2_sd15_T/vary_T_sd_15_1_2.txt", "rb") as fp:
    Ts = pickle.load(fp) # len = 2sd_R * 1 * 3_target * (3, 6)

In [10]:
# Combine results: 6[sd_R], 4 [tp], 3[metric], 6[method]

Ts = Ts + Ts1[1:] + Ts2[:3]
Ts = arr(Ts)

In [ ]:
# Combine results: 6[sd_R], 4 [tp], 3[metric], 6[method]
# Combine results
res = []
for i in range(len(res1)):
    res.append(res1[i])
    res.append(res2[i])
Ts = arr(Ts)

In [11]:
# save
from numpy import savetxt
file_name_pre = '/Users/mac/Google Drive/CausalMARL/res_MARL/2_sd15_T/res_sd_'

arr(Ts).shape
metrics = ["bias", "std", "MSE"]
for j in range(3):
    metric = Ts[:, :, j, :]
    print(metric.shape)
    for i in range(metric.shape[0]): # each sd_R [x-axis]
        savetxt(file_name_pre + str(metrics[j]) + str(i + 1) + ".txt", metric[i, :, :], delimiter=',')

(8, 4, 6)
(8, 4, 6)
(8, 4, 6)
